# Spot-check EEG classification models based on all features
In this script we spot-check different EEG classification models to clasify different epochs of one subject into resting state, left hand movement and right hand movement using leave one subject out cross-validation, performing a correlation based features selection.

## Import libraries

In [ ]:
import os
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from matplotlib import pyplot
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

## Create some functions

In [ ]:
# create a dict of some models to evaluate them {key:model name} {values:model object}
def define_models(models=dict()) :
    models['logistic'] = LogisticRegression()
    models['lda'] = LinearDiscriminantAnalysis()
    models['qda'] = QuadraticDiscriminantAnalysis()
    models['svcl'] = SVC(kernel='linear',probability=True)
    models['svmp'] = SVC(kernel='poly',probability=True)
    # non-linear models
    models['cart'] = DecisionTreeClassifier()
    models['knn'] = KNeighborsClassifier()  
    # ensemble models
    models['ada'] = AdaBoostClassifier()
    models['rf'] = RandomForestClassifier()
    models['mlp'] = MLPClassifier()
    return models

In [ ]:
# evaluate a single model
def evaluate_model(X, y, model, subjects_train):
    # evaluate model
    scores = dict()
    scores['accuracy'] = []
    scores['roc_auc'] = []
    scores['f1'] = []
    for subject in subjects_train :
        train_idx = subjects_idx_train != subject
        X_training = X[train_idx.values]
        y_training = y[train_idx.values]
        val_idx = subjects_idx_train == subject
        X_val = X[val_idx.values]
        y_val = y[val_idx.values]
        #train the model
        model.fit(X_training,y_training)
        y_pred = model.predict(X_val)
        y_prob = model.predict_proba(X_val)
        scores['accuracy'].append(accuracy_score(y_val,y_pred))
        scores['roc_auc'].append(roc_auc_score(y_val,y_prob,multi_class='ovo',labels=[0,1,2]))
        scores['f1'].append(f1_score(y_val,y_pred,average='weighted'))
    print(scores)
    return scores

In [ ]:
# evaluate a dict of models {name:object}, returns {name:score}
def evaluate_models(X, y, models, subjects_train):
    results = dict()
    metric='accuracy'
    for name, model in models.items():
        scores = evaluate_model(X, y, model, subjects_train)
        results[name] = scores
        print(name,'evaluated')
    return results

In [ ]:
#print and plot the results of the models in order (from the best one to the worse one)
def summarize_results(results):
    # create a list of (name, mean(scores)) tuples
    mean_accuracy_scores = dict()
    mean_auc_scores = dict()
    mean_f1_scores = dict()
    for k,v in results.items() :
        mean_accuracy_scores.update({k : mean(v['accuracy'])})
        mean_auc_scores.update({k : mean(v['roc_auc'])})
        mean_f1_scores.update({k : mean(v['f1'])})
    # sort tuples by mean score
    ordered_accuracy = sorted(mean_accuracy_scores.items(), key=lambda x: x[1])
    # reverse for descending order (from the best one to the worse one)
    ordered_accuracy = list(reversed(ordered_accuracy))
    # retrieve the top for summarization
    names = [x[0] for x in ordered_accuracy]
    accuracy_scores = [results[x]['accuracy'] for x in names]
    auc_scores = [results[x]['roc_auc'] for x in names]
    f1_scores = [results[x]['f1'] for x in names]
    # print the top 
    print()
    for i in range(len(results)):
        name = names[i]
        mean_accuracy_score = mean_accuracy_scores[name]
        mean_auc_score = mean_auc_scores[name]
        mean_f1 = mean_f1_scores[name]
        print('Rank=%d, Name=%s, Accuracy=%.3f, roc_auc=%.3f, f1=%.3f' % (i+1, name, mean_accuracy_score, mean_auc_score, mean_f1))
    # boxplot for the top n
    print(mean_accuracy_score, names)
    plt.figure()
    plt.boxplot(accuracy_scores, labels=names)
    _, labels = pyplot.xticks()
    pyplot.setp(labels, rotation=90)
    plt.title('Accuracy scores')
    plt.figure()
    plt.boxplot(auc_scores, labels=names)
    _, labels = pyplot.xticks()
    pyplot.setp(labels, rotation=90)
    plt.title('ROC-AUC scores')
    plt.figure()
    plt.boxplot(f1_scores, labels=names)
    _, labels = pyplot.xticks()
    pyplot.setp(labels, rotation=90)
    plt.title('F1 scores')

## Read features and labels from all subjects

In [ ]:
#read subjects files and merge them
files = os.listdir('Subjects data')
features_all = pd.read_csv('Subjects data/' + files[0], encoding='latin1')
files.pop(0)
for f in files :
    features_subject = pd.read_csv('Subjects data/' + f, encoding='latin1')
    frames = [features_all, features_subject]
    features_all = pd.concat(frames)
#features_all.index = [features_all['Subject'], features_all['Epoch']]
subjects_idx = features_all['Subject'].values
#features_all = features_all.drop(['Subject','Epoch'], axis = 1)
features_all

## Split the data into training and test

In [ ]:
#Select randomly subjects for train and test
subjects = np.unique(subjects_idx)
subjects_train = [15, 12, 21, 4, 10, 44, 3, 18, 29, 20, 7, 19, 11, 2, 28, 5, 8, 16, 6]
subjects_test = [1, 13, 17, 14, 9]
subjects_train, subjects_test

In [ ]:
train_idx = features_all['Subject'] == subjects_train[0]
features_train = features_all[train_idx]
subjects_train2 = np.delete(subjects_train, 0, axis=None)
for subject in subjects_train2 :
    train_idx = features_all['Subject'] == subject
    frames = [features_train, features_all[train_idx]]
    features_train = pd.concat(frames)
subjects_idx_train = features_train['Subject']
features_train

In [ ]:
features_train.index = [features_train['Subject'], features_train['Epoch']]
X_train = features_train.drop(['Subject','Epoch','Label'], axis = 1)
X_train

In [ ]:
Y_train = features_train['Label']
Counter(Y_train)

## Select the most highly correlated features with the target variable

Select the features that have a correlation higher than 0.5 with the target variable

In [ ]:
#Using Pearson Correlation
f = plt.figure(figsize=(12,10))
features = features_train.drop(['Subject','Epoch'], axis=1)
cor = features.corr()
plt.matshow(cor, fignum=f.number)
plt.xticks(range(features.shape[1]), features.columns, fontsize=14, rotation=45)
plt.yticks(range(features.shape[1]), features.columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16)

In [ ]:
#Correlation with output variable
cor_target = abs(cor['Label'])
#Selecting highly correlated features
relevant_features = cor_target[cor_target > 0.2]
selected_features = relevant_features[0:len(relevant_features)-1].index
selected_features

In [ ]:
X_train = X_train[selected_features]
X_train

## Spot-check algorithms standardizing data

Preprocess the features matrix

In [ ]:
X_train = X_train[selected_features]
standardizer = StandardScaler()
X_scaled_train = standardizer.fit_transform(X_train)
X_scaled_train.shape

In [ ]:
Counter(Y_train)

In [ ]:
# get model list
models = define_models()

In [ ]:
# evaluate models
results = evaluate_models(X_scaled_train, Y_train, models, subjects_train)
# summarize results
summarize_results(results)

Standardizing the data we obtain that the best three models are Logistic, LDA and SCVL with mean estimated accuracies of 0.509, 0.504 and 0.494 respectively. Let's try to normalize the standardized dataset to see if we can improve the metrics.

## Spot-check algorithms standardizing and normalizing data
Preprocess the training features matrix

In [ ]:
standardizer = StandardScaler()
X_scaled_train = standardizer.fit_transform(X_train)
normalizer = MinMaxScaler(feature_range=(0,1))
X_scaled_train = normalizer.fit_transform(X_scaled_train)
X_scaled_train.shape

In [ ]:
# evaluate models
results = evaluate_models(X_scaled_train, Y_train, models, subjects_train)
# summarize results
summarize_results(results)

## Delete features that are highly correlated with each other

In [ ]:
#Using Pearson Correlation
f = plt.figure(figsize=(12,10))
cor = X_train.corr()
plt.matshow(cor, fignum=f.number)
plt.xticks(range(X_train.shape[1]), X_train.columns, fontsize=14, rotation=45)
plt.yticks(range(X_train.shape[1]), X_train.columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16)

In [ ]:
correlated_features = []
for i in range(len(cor.columns)):
    for j in range(i):
        if abs(cor.iloc[i, j]) > 0.9:
            colname = cor.columns[i]
            correlated_features.append(colname)
correlated_features = list(dict.fromkeys(correlated_features))
correlated_features

In [ ]:
X_train = X_train.drop(correlated_features, axis=1)
X_train

In [ ]:
X_train.columns

In [ ]:
#Using Pearson Correlation
f = plt.figure(figsize=(12,10))
cor = X_train.corr()
plt.matshow(cor, fignum=f.number)
plt.xticks(range(X_train.shape[1]), X_train.columns, fontsize=14, rotation=45)
plt.yticks(range(X_train.shape[1]), X_train.columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Correlation Matrix', fontsize=16)

In this case we obtain that performing dimensionality reduction with PCA does not improve the model performance. Maybe with other dimensionality reduction/feature selection techniques? From the moment, let's try to perform PCA, standardize and normalize the data.

## Spot-check standardizing the data
Preprocess the training features matrix

In [ ]:
standardizer = StandardScaler()
X_scaled_train = standardizer.fit_transform(X_train)
X_scaled_train.shape

In [ ]:
# evaluate models
results = evaluate_models(X_scaled_train, Y_train, models, subjects_train)
# summarize results
summarize_results(results)

## Spot-check standardizing and normalizing the data

In [ ]:
standardizer = StandardScaler()
scaler = MinMaxScaler(feature_range=(0,1))
X_scaled_train = standardizer.fit_transform(X_train)
X_scaled_train = scaler.fit_transform(X_scaled_train)
X_scaled_train.shape

In [ ]:
# evaluate models
results = evaluate_models(X_scaled_train, Y_train, models, subjects_train)
# summarize results
summarize_results(results)